# UUID Related Types

We often use UUIDs as keys for our data (often in databases).

Python has several UUID related functions, the most common of which is UUID4.

In [1]:
from uuid import uuid4

In [2]:
uuid4()

UUID('35b4b12b-8202-483c-9635-84dee6cef719')

We can easily specify fields to be of that type:

In [3]:
from pydantic import BaseModel, Field, UUID4, ValidationError

In [4]:
class Person(BaseModel):
    id: UUID4

In [5]:
p = Person(id=uuid4())
p

Person(id=UUID('ffc70feb-3836-4300-9939-3cdb93e2c9dd'))

And when serialize the data:

In [6]:
p.model_dump()

{'id': UUID('ffc70feb-3836-4300-9939-3cdb93e2c9dd')}

In [7]:
p.model_dump_json()

'{"id":"ffc70feb-3836-4300-9939-3cdb93e2c9dd"}'

Now, how about setting a default value for `id`?

Can we just do this?

In [8]:
class Person(BaseModel):
    id: UUID4 = uuid4()

In [9]:
p1 = Person()
p1

Person(id=UUID('8d42d71d-e768-4533-a986-36bba0d9c744'))

In [10]:
p2 = Person()
p2

Person(id=UUID('8d42d71d-e768-4533-a986-36bba0d9c744'))

See the problem there? We are getting the same UUID4 for different instances.

We'll come back to this in greater detail, but let's look at it quickly here.

The other thing that you may not like, is using `id` as a field name (and even if you don't mind, linters will!)

So, we should use an alias - to do so we'll have to use a `Field` object, and we can see if using `default` in the `Field` will maybe fix the issue:

In [11]:
class Person(BaseModel):
    id_: UUID4 = Field(alias="id", default=uuid4())

In [12]:
p1 = Person()
p1

Person(id_=UUID('eb6aa05c-79fe-4a77-b2fc-f58042c5f57d'))

In [13]:
p2 = Person()
p2

Person(id_=UUID('eb6aa05c-79fe-4a77-b2fc-f58042c5f57d'))

So, using a `Field` default did not fix the problem.

The issue is the same as using using a function call to generate a default value for a function parameter.

The default gets created once when the function is compiled (created). Same with `Field` - it gets created once when the model is compiled, hence why we see the same uuid.

What we need is a way to have that default generated **every time** an instance is created.

We can do this by using the `default_factory` argument in `Field`.

This allows us to specify a callable, that Pydantic will call for every instance that needs to generate a default.

Let's try it:

In [14]:
class Person(BaseModel):
    id_: UUID4 = Field(alias="id", default_factory=uuid4)

Note that we specify a callable for `default_factory`, we do **not** call the function. Pydantic will call it when it needs to generate a new default value.

In [15]:
p1 = Person()
p1

Person(id_=UUID('21dbd0d0-9a2e-4126-a06a-b2429aa05112'))

In [16]:
p2 = Person()
p2

Person(id_=UUID('a70f8130-def5-4e22-ad30-ec3a602c62cd'))

Success!

Pydantic supports other UUID types as well, such as UUID1, UUID3 and UUID5.